# 04 Walk-Forward Validation

Stability of recession prediction across time.


## Table of Contents
- [Walk-forward splits](#walk-forward-splits)
- [Metric stability](#metric-stability)
- [Failure analysis](#failure-analysis)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Classification notebooks turn the recession label into a **probability model**.
You will learn how to evaluate rare-event prediction and how to choose thresholds intentionally.


## Prerequisites (Quick Self-Check)
- Completed Part 02 (regression basics) or equivalent.
- Comfort interpreting probabilities and trade-offs (false positives vs false negatives).

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Reporting only accuracy on imbalanced data.
- Using threshold=0.5 by default without considering costs.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/03_classification/04_walk_forward_validation.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/03_classification/04_walk_forward_validation.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Evaluate recession prediction stability over time using walk-forward validation.

Walk-forward answers the question:
- "Does my model work in *multiple eras*, or only in the era I trained on?"



## Primer: sklearn pipelines (how to avoid preprocessing leakage)

Pipelines prevent a common mistake: fitting preprocessing (scaling, imputation) using information from the test set.

### Why pipelines exist (in one sentence)

> A `Pipeline` ensures that transformations are fit on training data only, then applied to test data.

### The key APIs

- `fit(X, y)`: learn parameters (scaler mean/std, model weights) from training.
- `transform(X)`: apply learned transform to new data.
- `fit_transform(X, y)`: convenience for training data only.

### Minimal pattern (classification)

```python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
  ("scaler", StandardScaler()),
  ("model", LogisticRegression(max_iter=5000)),
])

# clf.fit(X_train, y_train)
# y_prob = clf.predict_proba(X_test)[:, 1]
```

**Expected output / sanity check**
- you never call `scaler.fit` on the full dataset
- you split by time first, then fit the pipeline on train

### Mini demo: the leakage you’re avoiding (toy)

```python
import numpy as np
from sklearn.preprocessing import StandardScaler

rng = np.random.default_rng(0)
X_train = rng.normal(loc=0.0, scale=1.0, size=(100, 1))
X_test  = rng.normal(loc=2.0, scale=1.0, size=(25, 1))

# WRONG: fit on train+test (leaks the future)
sc_wrong = StandardScaler().fit(np.vstack([X_train, X_test]))
X_test_wrong = sc_wrong.transform(X_test)

# RIGHT: fit on train only
sc_right = StandardScaler().fit(X_train)
X_test_right = sc_right.transform(X_test)

print("test mean after wrong scaling:", float(X_test_wrong.mean()))
print("test mean after right scaling:", float(X_test_right.mean()))
```

### Common pitfalls

- Splitting after preprocessing (leakage).
- Using random splits on time-indexed data (temporal leakage).
- Forgetting `ColumnTransformer` for mixed numeric/categorical columns (if needed).


<a id="walk-forward-splits"></a>
## Walk-forward splits

### Goal
Generate a sequence of train/test splits that move forward through time.



### Your Turn (1): Load data and define X/y


In [ ]:
import pandas as pd

path = PROCESSED_DIR / 'macro_quarterly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)

y_col = 'target_recession_next_q'
x_cols = ['T10Y2Y_lag1', 'UNRATE_lag1', 'FEDFUNDS_lag1', 'INDPRO_lag1', 'RSAFS_lag1']
df_m = df[[y_col] + x_cols].dropna().copy()

X = df_m[x_cols]
y = df_m[y_col].astype(int)
X.shape, y.mean()



### Your Turn (2): Generate walk-forward splits


In [ ]:
from src.evaluation import walk_forward_splits

# TODO: Choose split settings.
# initial_train_size: first training window size
# test_size: number of quarters per fold
splits = list(walk_forward_splits(len(df_m), initial_train_size=40, test_size=8))
len(splits), splits[0]



<a id="metric-stability"></a>
## Metric stability

### Goal
Fit the same model on each fold and track metrics across time.



### Your Turn (1): Evaluate a model across folds


In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from src.evaluation import classification_metrics

rows = []
for sp in splits:
    X_tr = X.iloc[sp.train_slice]
    y_tr = y.iloc[sp.train_slice]
    X_te = X.iloc[sp.test_slice]
    y_te = y.iloc[sp.test_slice]

    clf = Pipeline([
        ('scaler', StandardScaler()),
        ('model', LogisticRegression(max_iter=5000)),
    ])
    clf.fit(X_tr, y_tr)
    p = clf.predict_proba(X_te)[:, 1]
    m = classification_metrics(y_te.to_numpy(), p, threshold=0.5)

    rows.append({
        'train_end': X_tr.index.max(),
        'test_end': X_te.index.max(),
        **m,
    })

wf = pd.DataFrame(rows).set_index('test_end')
wf.head()



### Your Turn (2): Plot metrics over time


In [ ]:
import matplotlib.pyplot as plt

# TODO: Plot roc_auc, pr_auc, and brier over time.
...



<a id="failure-analysis"></a>
## Failure analysis

### Goal
Identify which eras the model struggles with and investigate why.



### Your Turn (1): Find the worst fold


In [ ]:
# TODO: Pick a metric and find the worst fold.
# Example: lowest PR-AUC
worst = wf.sort_values('pr_auc').head(1)
worst



### Your Turn (2): Inspect features in that era


In [ ]:
# TODO: Look at feature distributions in the worst fold.
# Compare to a better-performing fold.
...



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Walk-forward splits</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_walk_forward_validation — Walk-forward splits
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from src import evaluation

df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True).dropna()
target = 'target_recession_next_q'
drop_cols = {target, 'recession', 'GDPC1', 'gdp_growth_qoq', 'gdp_growth_qoq_annualized', 'gdp_growth_yoy'}
X = df[[c for c in df.columns if c not in drop_cols]].astype(float)
y = df[target].astype(int).to_numpy()

clf = Pipeline([('scaler', StandardScaler()), ('m', LogisticRegression(max_iter=5000))])

metrics = []
for split in evaluation.walk_forward_splits(len(df), initial_train_size=20, test_size=4, step_size=2):
    clf.fit(X.iloc[split.train_slice], y[split.train_slice])
    p = clf.predict_proba(X.iloc[split.test_slice])[:,1]
    m = evaluation.classification_metrics(y[split.test_slice], p)
    metrics.append(m)

pd.DataFrame(metrics).head()
```

</details>

<details><summary>Solution: Metric stability</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_walk_forward_validation — Metric stability
# Plot ROC-AUC/PR-AUC over folds and inspect variance.
```

</details>

<details><summary>Solution: Failure analysis</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_walk_forward_validation — Failure analysis
# Identify folds with worst metrics and map to dates.
```

</details>

